# Load the data for Jason's slow bike line tracking tasks

In [ ]:
import bicycledataprocessor as bdp

In [ ]:
dataset = bdp.DataSet()

These are the slow bike line tracking tasks for Jason:

In [ ]:
run_ids = ['00{}'.format(n) for n in range(814, 820)]
run_ids

In [ ]:
runs = [bdp.Run(run_id, dataset, filterFreq=10.0) for run_id in run_ids]

# Check sensor battery voltages

Let's have a look at the voltage levels for every one of the runs.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
fig, axes = plt.subplots(len(runs), figsize=(6, 10))
for run, ax in zip(runs, axes):
    run.rawSignals['ThreeVolts'].plot(ax=ax)
    run.rawSignals['FiveVolts'].plot(ax=ax)
    run.rawSignals['PushButton'].plot(ax=ax)
    ax.set_ylabel('Voltage')
    ax.legend(['ThreeVolts', 'FiveVolts', 'PushButton'])
plt.tight_layout()

I'm not completely sure but the push button voltage should be closer to zero when the button is not pushed. It doesn't seem to be causing any issues, but seems odd.

In [ ]:
run.taskSignals['FiveVolts'].mean()

In [ ]:
run.taskSignals['FiveVolts'].std()

In [ ]:
run.taskSignals['ThreeVolts'].mean()

In [ ]:
run.taskSignals['ThreeVolts'].std()

# Check the forward speed variation

Below is a comparison of the speed maintained during the task. All of the runs bu 814, I slowed down at a more-or-less constant rate. It is very difficult to get the cruise control set a low speeds. I think I set it for the last few runs, but there was still the same drop in speed that we see 815, where the note indicates that there is was definitely no cruise control. This may be a problem beause handling varies so much with speed. We need to at least get a constant speed.

In [ ]:
fig, ax = plt.subplots(1)
for run in runs:
    run.taskSignals['ForwardSpeed'].plot(ax=ax)
ax.legend(run_ids)
ax.set_title('')
plt.tight_layout()

A quick comparison of the forward speeds during each run can be done with box plots. The blue line and shaded region shows the mean and standard deviation among the runs.

In [ ]:
plt.figure()
plt.boxplot([run.taskSignals['ForwardSpeed'] for run in runs], labels=run_ids)
plt.ylabel('ForwardSpeed [meter/second]')
plt.xlabel('RunID');

In [ ]:
import numpy as np
mean_speed_of_runs = np.mean([run.taskSignals['ForwardSpeed'].mean() for run in runs])
mean_line = plt.axhline(mean_speed_of_runs);

In [ ]:
std_speed_of_runs = np.std([run.taskSignals['ForwardSpeed'].mean() for run in runs])
plt.fill_between(plt.gca().get_xlim(), mean_speed_of_runs - std_speed_of_runs,
                 mean_speed_of_runs + std_speed_of_runs, alpha=0.5);

# Steer angle variation

Now let's compare the steer angle of each of these runs.

In [ ]:
fig, axes = plt.subplots(len(run_ids), figsize=(8, 12))
for run, ax in zip(runs, axes):
    run.taskSignals['SteerAngle'].plot(ax=ax)
    ax.set_ylim((-0.25, 0.25))
plt.tight_layout()

Here I check whether the standard deviation in the steering angle correlates to the weight position. It seems to, except for the repeat of the position 4 run, which ends up being the worst in terms of steering. The boxplot is sorted left to right by the standard deviation in the steering angle.

In [ ]:
# maps the weight position to the run
pos_map = {814: 1, 815: 4, 816: 4, 817: 2, 818: 6, 819: 7}

In [ ]:
plt.figure(figsize=(8, 6))
steer_angles = sorted([(run.metadata['RunID'], run.taskSignals['SteerAngle']) for run in runs],
                      key=lambda tup: tup[1].std())
plt.boxplot([s[1] for s in steer_angles], labels=['{}, pos={}'.format(s[0], pos_map[s[0]]) for s in steer_angles])
plt.ylabel('SteerAngle [radian]')
plt.xlabel('RunID');

# Wheel tracks

Here we should check whether the wheel tracks were ever wider than the lane width to determine if the performance was adequate. We can also check the standard deviation to see how well the rider stayed on the line.

In [ ]:
fig, axes = plt.subplots(len(run_ids), figsize=(8, 12))
for run, ax in zip(runs, axes):
    run.plot_wheel_contact(ax=ax)
    # scott says the lane width is 18 inches
    ax.axhline(run.taskSignals['LateralRearContact'].mean() + .2286, color='black')
    ax.axhline(run.taskSignals['LateralRearContact'].mean() - .2286, color='black')
    ax.set_ylim((-0.25, 0.25))
plt.tight_layout()

Below we see a comparison of the variation in the lateral location of the rear wheel contact point. The boxes are sorted by standard deviation and not much correlation is seen with the weight position.

In [ ]:
plt.figure(figsize=(8, 6))
steer_angles = sorted([(run.metadata['RunID'], run.taskSignals['LateralRearContact']) for run in runs],
                      key=lambda tup: tup[1].std())
plt.boxplot([s[1] for s in steer_angles], labels=['{}, pos={}'.format(s[0], pos_map[s[0]]) for s in steer_angles])
plt.ylabel('LateralRearContact [meters]')
plt.xlabel('RunID');

Below we see a comparison of the variation in the lateral location of the rear wheel contact point. The boxes are sorted by standard deviation and there is evidence of correlation with the weight position, except for run 817.

In [ ]:
plt.figure(figsize=(8, 6))
steer_angles = sorted([(run.metadata['RunID'], run.taskSignals['LateralFrontContact']) for run in runs],
                      key=lambda tup: tup[1].std())
plt.boxplot([s[1] for s in steer_angles], labels=['{}, pos={}'.format(s[0], pos_map[s[0]]) for s in steer_angles])
plt.ylabel('LateralFrontContact [meters]')
plt.xlabel('RunID');